# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents_2.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

5978799

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

absence_&


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

1

In [14]:
print(list(bigram_phrases.vocab.keys())[:100])

['v', 'i', 'v_i', 'l', 'i_l', 'l_l', 'e', 'l_e', 'de', 'e_de', 'bruxelles', 'de_bruxelles', 'bulletin', 'bruxelles_bulletin', 'ires', 'bulletin_ires', '8eanas', 'ires_8eanas', 'dl', '8eanas_dl', '!', 'dl_!', 'conseil', 'communal', 'conseil_communal', 'annee', 'communal_annee', '1847', 'annee_1847', '.', '1847_.', 'au', 'ville', 'au_ville', 'ville_de', 'b', 'de_b', 'r', 'b_r', 'u', 'r_u', 'x', 'u_x', 'x_e', 'e_l', 's', 'e_s', 's_.', 'bulletin_conseil', 'aes', 'conseil_aes', 'seances', 'aes_seances', 'seances_communal', 'communal_.', ',', 'bruxelles_,', 'imprimerie', ',_imprimerie', 'd', 'imprimerie_d', 'd_e', 'j', 'e_j', 'j_.', 'h', 'h_.', '._b', 'r_i', 'a', 'i_a', 'a_r', 'r_d', 'd_,', 'rite', ',_rite', 'n', 'rite_n', 'n_e', 'e_u', 'u_v', 'v_e', 'e_,', '3', ',_3', '1', '3_1', '1_,', 'faubourg', ',_faubourg', 'faubourg_de', 'de_n', 'n_a', 'm', 'a_m', 'm_u', 'u_r', 'r_,', ',_1', '84']


Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

In [15]:
%time bigram_phrases[sentences]

CPU times: user 7.29 ms, sys: 0 ns, total: 7.29 ms
Wall time: 14.4 ms


### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [16]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

In [17]:
%time bigram_phraser[sentences]

CPU times: user 12.5 ms, sys: 88 µs, total: 12.6 ms
Wall time: 16.1 ms


### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [18]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [19]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:100])

## Entrainement d'un modèle Word2Vec sur ce corpus (modèle 10)

In [17]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=2, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 28min 15s, sys: 25.5 s, total: 28min 41s
Wall time: 14min 13s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [18]:
outfile = f"../data/bulletins_10.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [19]:
model = Word2Vec.load("../data/bulletins_10.model")

### Imprimer le vecteur d'un terme

In [20]:
model.wv["bruxelles"]

array([ 1.1260105 , -2.1725283 ,  0.4175255 , -0.69901973,  0.97302616,
       -2.6768591 , -1.5538032 ,  2.8593185 , -3.2508225 , -4.498826  ,
       -1.0544505 ,  4.9645147 , -2.5055726 ,  4.3030825 ,  4.922113  ,
       -0.27940458,  2.6213033 , -2.7516031 ,  3.5029454 ,  1.9816391 ,
       -2.734784  , -3.1897314 , -1.4461392 ,  2.9164798 , -3.4263446 ,
        3.1776385 ,  4.9700117 ,  1.8515121 ,  3.8447385 ,  1.0398239 ,
       -1.0671779 ,  1.2635777 ], dtype=float32)

### Calculer la similarité entre deux termes

In [21]:
model.wv.similarity("boucher", "boulanger")

0.88451433

### Chercher les mots les plus proches d'un terme donné

In [22]:
model.wv.most_similar("bruxelles", topn=10)

[('laeken', 0.8489513397216797),
 ('paris', 0.8009144067764282),
 ('liege', 0.7990047335624695),
 ('nivelles', 0.7926765084266663),
 ('considere_comme_officiel', 0.7591863870620728),
 ('gand', 0.7544216513633728),
 ('rruxelles', 0.7373996376991272),
 ('huy', 0.7307796478271484),
 ('louvain', 0.7209203839302063),
 ('binche', 0.7185885310173035)]

### Faire des recherches complexes à travers l'espace vectoriel

In [23]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('huy', 0.8321443200111389), ('paris', 0.8168970346450806), ('manchester', 0.8040940761566162), ('verviers', 0.798650860786438), ('norvege', 0.7958981394767761), ('lille', 0.7932437658309937), ('gand', 0.7927877902984619), ('prague', 0.7915379405021667), ('zurich', 0.7848337292671204), ('tournai', 0.7804679870605469)]


### Modèle 11: Taille du vecteur = 100 (vs 32)

In [24]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=5, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=2, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 39min 11s, sys: 36.1 s, total: 39min 47s
Wall time: 20min 7s


In [25]:
outfile = f"../data/bulletins_11.model"
model.save(outfile)

In [26]:
model = Word2Vec.load("../data/bulletins_11.model")

In [27]:
model.wv["bruxelles"]

array([-1.1539743e+00,  1.8585192e+00, -1.6800332e+00, -4.4222820e-01,
        1.7482232e+00,  3.9750993e+00, -2.6492040e+00, -5.3296113e+00,
        1.5155109e+00, -1.9158862e+00,  5.2028531e-01,  9.3181378e-01,
        1.2822436e+00, -8.5219488e-02,  3.4232870e-02,  1.7136720e-01,
        1.9729372e+00,  3.5099645e+00, -9.8435052e-02,  1.2754691e+00,
        4.1667604e+00, -3.1279242e+00, -1.8638443e+00,  2.1328425e+00,
        2.6962016e+00,  9.5783979e-01,  4.8206758e-01, -1.3146660e-01,
        9.5839614e-01,  1.3258016e+00,  6.5184885e-01,  2.2936983e+00,
        1.2322680e+00,  5.0789362e-01, -5.1532584e-01,  1.2043320e+00,
        1.1132662e+00, -3.3199067e+00, -1.6523521e+00, -5.9322041e-01,
        2.6820681e+00,  4.6180341e-01, -2.8952336e+00,  2.0132332e+00,
        1.1390903e+00, -4.2447958e+00, -7.7328593e-01, -1.0703262e+00,
        2.4343159e+00,  1.5587091e-03, -2.3235846e+00, -3.4852853e-01,
       -4.2528143e+00, -1.6277497e+00, -1.3276261e+00, -1.5694494e+00,
      

In [28]:
model.wv.similarity("boucher", "boulanger")

0.74020714

In [29]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.7136662602424622),
 ('nivelles', 0.6468287110328674),
 ('prague', 0.6252561807632446),
 ('liege', 0.6158735752105713),
 ('paris', 0.6153978705406189),
 ('bruxelle', 0.6121673583984375),
 ('lille', 0.6068112850189209),
 ('laeken', 0.6047728657722473),
 ('gand', 0.6014081835746765),
 ('lhotel', 0.5876725912094116)]

In [30]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('prague', 0.6978469491004944), ('paris', 0.6645860075950623), ('lille', 0.6635715961456299), ('tournai', 0.6521830558776855), ('londres', 0.6501026749610901), ('liege', 0.6492911577224731), ('gand', 0.6386280655860901), ('huy', 0.6197343468666077), ('milan', 0.5927150845527649), ('hambourg', 0.5915898084640503)]


### Modèle 12: Vecteur 100 + élargissement de la fenêtre à 7

In [31]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=7, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 49min 7s, sys: 1min 11s, total: 50min 19s
Wall time: 16min 36s


In [32]:
outfile = f"../data/bulletins_12.model"
model.save(outfile)

In [33]:
model = Word2Vec.load("../data/bulletins_12.model")

In [34]:
model.wv["bruxelles"]

array([-2.5097110e+00, -1.6216507e+00, -1.7737223e+00,  2.2958612e+00,
        1.4272438e+00, -5.5398822e-02, -4.7675767e+00, -3.9126141e+00,
        3.2183037e+00, -4.2289095e+00, -8.1615108e-01,  3.6222725e-03,
        1.5715544e+00, -9.6815938e-01,  3.7148803e-01,  1.8808427e+00,
        4.0873914e+00,  3.8944528e+00,  8.0108821e-01,  1.5176334e+00,
        4.3372049e+00, -4.3116035e+00, -1.1436946e+00,  3.0126836e+00,
        3.1584575e+00, -1.3725473e+00, -2.1337962e+00,  1.9179517e-01,
        5.1859361e-01,  1.1316208e+00, -1.0905432e+00,  2.4017324e+00,
        4.8845053e-01,  5.2066278e-01, -1.1646289e+00, -2.3857501e-01,
       -1.6562866e+00,  2.9832468e+00, -1.7789822e+00,  2.7195344e+00,
       -1.4348168e+00,  2.2810922e+00, -1.8511634e+00,  4.1115832e-01,
        1.1041604e-01, -3.4578316e+00, -4.9020991e+00,  3.4461372e-02,
        1.0478247e+00,  1.3229064e+00, -2.8644516e+00,  7.5171903e-02,
        1.2130132e+00, -7.6969671e-01, -1.7908067e+00, -3.7262318e+00,
      

In [35]:
model.wv.similarity("boucher", "boulanger")

0.74992484

In [36]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.6743203401565552),
 ('liege', 0.6394780874252319),
 ('prague', 0.6360003352165222),
 ('bruxelle', 0.6343247890472412),
 ('xelles', 0.6240631341934204),
 ('bruxe', 0.6227555871009827),
 ('nivelles', 0.6150801181793213),
 ('lille', 0.612572193145752),
 ('paris', 0.6059279441833496),
 ('bruxeles', 0.5985421538352966)]

In [37]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('prague', 0.6655336022377014), ('londres', 0.6648284792900085), ('paris', 0.6592709422111511), ('lille', 0.658177375793457), ('liege', 0.6438037157058716), ('strasbourg', 0.6386304497718811), ('huy', 0.6232372522354126), ('verviers', 0.6213777661323547), ('gand', 0.6208687424659729), ('tournai', 0.610344409942627)]


In [38]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('magistrat', 0.5965210199356079), ('prince', 0.5952725410461426), ('parlement', 0.5779935121536255), ('souverain', 0.5624872446060181), ('geste', 0.5579628348350525), ('peuple', 0.5476931929588318), ('gouvernement', 0.5424214005470276), ('citoyen', 0.5420892238616943), ('soldat', 0.5411161184310913), ('directoire_executif', 0.5341085195541382)]


In [39]:
print(model.wv.most_similar(positive=['france', 'belgique'], negative=['paris']))

[('russie', 0.6977651715278625), ('hollande', 0.6429075598716736), ('gique', 0.6396220922470093), ('prusse', 0.6370446681976318), ('grece', 0.6329469084739685), ('pologne', 0.6301029920578003), ('egypte', 0.6247028708457947), ('allemagne', 0.6204552054405212), ('prose', 0.6186758875846863), ('magne', 0.6132579445838928)]


### Modèle 13: Taille fenêtre 20 (vs 5) et vecteur 32 (vs 100)

In [40]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 53min 1s, sys: 1min 24s, total: 54min 26s
Wall time: 17min 25s


In [41]:
outfile = f"../data/bulletins_13.model"
model.save(outfile)

In [42]:
model = Word2Vec.load("../data/bulletins_13.model")

In [43]:
model.wv["bruxelles"]

array([-0.971476  ,  1.1585819 ,  1.9293963 ,  2.2696738 , -0.95015925,
        0.4804774 , -3.5447733 ,  0.19505647, -1.0813653 , -2.173142  ,
        2.5600793 , -3.197074  ,  1.8823612 , -2.4818788 ,  0.51100093,
       -0.05277269, -2.0051522 ,  4.0309167 ,  3.00355   , -1.6767248 ,
        7.8519526 , -1.6590991 ,  3.3218467 ,  1.8396266 ,  2.589873  ,
        4.742811  , -0.5411295 ,  1.102619  ,  3.304934  ,  0.37661466,
       -2.6051095 ,  1.3045704 ,  0.66989094, -1.9110194 ,  1.4860145 ,
       -0.6226967 ,  0.9053694 ,  0.94488925,  0.9938249 , -2.2563617 ,
        2.6645045 ,  2.2340987 ,  0.24985437, -0.72093827,  2.3941057 ,
       -0.21500607, -3.6347086 ,  1.2598578 ,  0.9621279 , -0.02723751,
       -3.3255062 ,  2.0855017 , -7.586353  ,  0.7574345 , -3.7482953 ,
        1.776607  ,  4.2794366 , -3.969426  ,  2.6658452 ,  4.8011866 ,
        3.8450189 ,  1.7982588 ,  0.03574454,  3.5566518 ,  2.6109102 ,
       -0.7125153 , -0.19396746,  3.6263819 ,  0.17875561, -8.23

In [44]:
model.wv["boucher"]

array([ 1.0531648 , -0.67007715, -0.8484522 , -0.3299128 ,  1.9245322 ,
       -1.0435863 ,  2.6561742 ,  0.35014355,  0.34272856, -0.05957036,
        0.13900189, -3.10177   , -1.219908  ,  0.29260308,  0.6749098 ,
        1.212267  , -0.36900538,  1.6241764 ,  1.6359512 , -0.54867625,
        2.4581873 , -0.0513159 , -2.1077986 , -2.5916808 ,  0.6399825 ,
       -1.9364533 ,  0.07702505, -1.4731607 ,  2.1370702 , -1.2038982 ,
       -0.18382151, -1.9336646 ,  1.35962   ,  1.2636005 ,  2.1745293 ,
       -1.0393125 , -1.5382365 , -0.4658787 ,  1.3203089 ,  1.1236529 ,
        0.8015843 , -0.19109268, -2.1660376 ,  0.2275303 , -0.48223156,
        0.16956599, -0.6738468 , -1.8383192 ,  0.51808244,  0.24323906,
       -1.7735149 , -1.1688193 , -0.42180204, -0.91761196,  2.64465   ,
        2.7966313 ,  1.0244737 ,  0.1243986 , -0.95424265, -0.74975544,
       -0.8377274 , -1.2847992 , -0.26673326,  1.1967674 , -2.9241085 ,
        1.2438351 ,  0.14159115,  1.4974174 , -0.41187888,  0.02

In [45]:
model.wv.similarity("boucher", "boulanger")

0.6937786

In [46]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.651600182056427),
 ('nivelles', 0.638759195804596),
 ('bruxelle', 0.6351412534713745),
 ('liege', 0.6297926902770996),
 ('xelles', 0.6212089657783508),
 ('bruxe', 0.6058027148246765),
 ('bruxeles', 0.5974170565605164),
 ('thotel', 0.5858594179153442),
 ('paris', 0.5826448798179626),
 ('laeken', 0.5800283551216125)]

In [47]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('liege', 0.6205630302429199), ('paris', 0.6186079382896423), ('lille', 0.617577850818634), ('verviers', 0.6161673069000244), ('londres', 0.6111919283866882), ('huy', 0.6109883189201355), ('tournai', 0.602641224861145), ('gand', 0.596588671207428), ('prague', 0.595768392086029), ('marseille', 0.5875338912010193)]


In [48]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('parlement', 0.6019852161407471), ('magistrat', 0.5870837569236755), ('prince', 0.5709598064422607), ('peuple', 0.5621649622917175), ('substitut', 0.5548692345619202), ('geste', 0.5506982803344727), ('roi_guillaume', 0.54563969373703), ('commissaire_special', 0.5449787974357605), ('episode', 0.5426084995269775), ('congres_national', 0.5395960807800293)]


In [49]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))

[('londres', 0.5867902040481567), ('gand', 0.5701483488082886), ('lille', 0.5545138120651245), ('strasbourg', 0.5509448647499084), ('manchester', 0.5442072749137878), ('liege', 0.5433911681175232), ('hambourg', 0.53736811876297), ('verviers', 0.5217142105102539), ('prague', 0.5190670490264893), ('amsterdam', 0.5169219970703125)]


### Modèle 14 : Vecteur 100 et fenêtre 13

In [50]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=13, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 57min 44s, sys: 1min 12s, total: 58min 57s
Wall time: 18min 45s


In [51]:
outfile = f"../data/bulletins_14.model"
model.save(outfile)

In [52]:
model = Word2Vec.load("../data/bulletins_14.model")

In [53]:
model.wv["bruxelles"]

array([-3.14808607e-01, -2.02222228e+00, -3.66169238e+00,  4.28409576e-01,
        2.02761793e+00,  1.08896542e+00, -1.80558741e-01,  1.29934645e+00,
        3.53410769e+00,  1.14933781e-01, -1.53659098e-02,  8.16034794e-01,
        3.74506402e+00, -8.76322314e-02,  2.50448203e+00,  3.27910256e+00,
        5.81417751e+00,  4.16030836e+00,  5.54160070e+00, -3.01619482e+00,
        7.16032982e+00, -3.53323293e+00, -5.94187307e+00,  5.98532581e+00,
        2.72803128e-01, -4.08051443e+00,  1.68708339e-01, -1.85459256e+00,
        7.18670666e-01,  4.21209540e-03,  1.61055648e+00,  8.72412562e-01,
        4.39906025e+00,  9.89009798e-01,  3.16437244e+00, -1.55942190e+00,
        3.88595653e+00,  1.07271934e+00, -1.27110615e-01, -1.23921764e+00,
       -6.21512413e+00,  2.40721464e+00, -2.61588335e+00,  6.43322170e-01,
       -1.15487814e+00, -5.11716557e+00, -2.07160139e+00,  1.46752000e+00,
        1.20237017e+00,  2.73816276e+00, -7.97673643e-01,  1.02384746e-01,
       -1.03925812e+00, -

In [54]:
model.wv.similarity("boucher", "boulanger")

0.6723962

In [55]:
model.wv.similarity("voiture", "carrosse")

0.31495377

In [56]:
model.wv.most_similar("bruxelles", topn=10)

[('xelles', 0.6420583724975586),
 ('bruxelle', 0.6356337070465088),
 ('manufacturiere', 0.6315798759460449),
 ('nivelles', 0.6286843419075012),
 ('liege', 0.6279160976409912),
 ('dette_active', 0.6258662939071655),
 ('rruxelles', 0.620442807674408),
 ('tournai', 0.584047257900238),
 ('bruxe', 0.5773397088050842),
 ('prague', 0.5762890577316284)]

In [57]:
model.wv.most_similar("boucher", topn=10)

[('charcutier', 0.7419087290763855),
 ('batelier', 0.7248253226280212),
 ('cabaretier', 0.7033025622367859),
 ('marchand', 0.6830555200576782),
 ('serrurier', 0.6813036799430847),
 ('menuisier', 0.6732695698738098),
 ('boulanger', 0.672396183013916),
 ('abatteur', 0.6655840277671814),
 ('meunier', 0.6591508984565735),
 ('tailleur', 0.6587807536125183)]

In [58]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('londres', 0.627091646194458), ('prague', 0.6158853769302368), ('liege', 0.6125531792640686), ('paris', 0.6043033599853516), ('manchester', 0.602205216884613), ('lille', 0.5956189632415771), ('ostende', 0.5921670794487), ('tournai', 0.5895686149597168), ('huy', 0.588439404964447), ('verviers', 0.5736773014068604)]


In [59]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('prince', 0.612407922744751), ('magistrat', 0.5882052779197693), ('souverain', 0.5804455876350403), ('parlement', 0.5735340714454651), ('congres_national', 0.5685485601425171), ('peuple', 0.5683119297027588), ('citoyen', 0.5606240630149841), ('homme_politique', 0.5573277473449707), ('geste', 0.5502994060516357), ('penseur', 0.5411233901977539)]


In [60]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('collegue', 0.5705028772354126), ('defenseur', 0.5656442642211914), ('ministre', 0.5545243620872498), ('scrupule', 0.542506217956543), ('geste', 0.5404203534126282), ('conseiller', 0.538261890411377), ('premier_ministre', 0.5310626029968262), ('prob', 0.5282989740371704), ('nistre', 0.5277909636497498), ('discours', 0.5143312811851501)]


In [61]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('rruxelles', 0.5320407152175903), ('manufacturiere', 0.5163587927818298), ('bruxelle', 0.5104468464851379), ('bruxeves', 0.4848758578300476), ('bruxelles_previent', 0.4795002043247223), ('baron_steens', 0.4618682861328125), ('nivelles', 0.45592978596687317), ('anversoise', 0.45487308502197266), ('bruxe', 0.45260101556777954), ('binche', 0.45215967297554016)]


In [62]:
print(model.wv.most_similar(positive=['bruxelles', 'belgique'], negative=['france']))

[('dette_active', 0.5938431024551392), ('rruxelles', 0.5544723272323608), ('bruxeves', 0.5492538213729858), ('lhotel', 0.5239477157592773), ('bruxelle', 0.5142613053321838), ('brussel_bruxelles_gemeenteblad', 0.5085204839706421), ('xelles', 0.5079637765884399), ('bruxe', 0.5027642250061035), ('jumeaux_naissances', 0.4880010783672333), ('immobilisations_propres', 0.4863151013851166)]


### Modèle 15 : 300 dimensions et élargissement de la fenêtre à 13 et entrainement ++

In [63]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=13, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 1h 50min 13s, sys: 4min 58s, total: 1h 55min 11s
Wall time: 32min 57s


In [64]:
outfile = f"../data/bulletins_15.model"
model.save(outfile)

In [65]:
model = Word2Vec.load("../data/bulletins_15.model")

In [66]:
model.wv["bruxelles"]

array([-1.6131113e+00, -1.7705723e+00,  1.5323797e+00, -1.0271399e+00,
       -1.3975497e+00,  1.2016973e+00,  5.1747388e-01, -4.9310511e-01,
       -4.8463640e-01,  3.3092864e+00,  1.5544190e+00,  3.7663591e+00,
        2.4313972e+00,  1.5103762e-01, -5.0827867e-01, -2.4846964e+00,
        1.9778758e+00,  3.1144278e+00,  1.2442329e+00,  1.1813360e+00,
        1.3347781e+00,  2.2994943e+00,  1.0616659e+00,  1.8465179e+00,
       -1.6029588e+00, -2.0999169e+00,  1.4347253e+00, -8.6572230e-02,
        2.5102215e+00, -3.3589727e-01,  3.8957560e-01, -4.4208968e-01,
        1.2430557e+00,  2.9331477e+00,  1.0393891e+00,  3.4847264e+00,
        7.7900916e-02,  3.5337523e-01,  4.1654860e-03,  8.0260283e-01,
       -3.1242816e+00,  1.0474235e+00,  4.0889068e+00,  1.4083492e+00,
        1.3219323e-02, -3.6417234e+00, -3.8513380e-01,  3.4908571e+00,
        3.2785473e+00,  2.0484619e-01,  1.8237715e+00,  2.9266566e-01,
        1.1467165e-04,  2.0547850e+00, -8.1822497e-01,  1.0596249e+00,
      

In [67]:
model.wv.similarity("boucher", "boulanger")

0.5587284

In [68]:
model.wv.similarity("voiture", "carrosse")

0.26498124

In [69]:
model.wv.most_similar("bruxelles", topn=10)

[('rruxelles', 0.5277640223503113),
 ('dette_active', 0.49114567041397095),
 ('liege', 0.48792389035224915),
 ('bruxelle', 0.4822578728199005),
 ('manufacturiere', 0.4812834858894348),
 ('nivelles', 0.4720197021961212),
 ('xelles', 0.4664071798324585),
 ('binche', 0.4439750015735626),
 ('verviers', 0.44394662976264954),
 ('tournai', 0.4434230625629425)]

In [70]:
model.wv.most_similar("boucher", topn=10)

[('charcutier', 0.5835238695144653),
 ('cordonnier', 0.5590786933898926),
 ('boulanger', 0.5587283372879028),
 ('menuisier', 0.5515034794807434),
 ('abatteur', 0.551483154296875),
 ('tailleur', 0.5508565306663513),
 ('cabaretier', 0.5470849871635437),
 ('serrurier', 0.5364952683448792),
 ('coiffeur', 0.5345790386199951),
 ('marchand', 0.5307872891426086)]

In [71]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('londres', 0.48951879143714905), ('liege', 0.4573286175727844), ('manchester', 0.4476049840450287), ('milan', 0.4397909343242645), ('ostende', 0.4391721785068512), ('prague', 0.4373452961444855), ('marseille', 0.4363729953765869), ('paris', 0.4332381784915924), ('verviers', 0.4286613166332245), ('lille', 0.42616918683052063)]


In [72]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('souverain', 0.4747852385044098), ('prince', 0.45544907450675964), ('citoyen', 0.4522494375705719), ('magistrat', 0.44988343119621277), ('peuple', 0.44894155859947205), ('congres_national', 0.43645983934402466), ('peuple_belge', 0.43318602442741394), ('geste', 0.43007296323776245), ('soldat', 0.42618703842163086), ('gouvernement', 0.4182727634906769)]


In [73]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('collegue', 0.4349214434623718), ('conseiller', 0.43413612246513367), ('geste', 0.43205609917640686), ('scrupule', 0.42409834265708923), ('citoyen', 0.4215727150440216), ('journaliste', 0.4145069718360901), ('confrere', 0.41157031059265137), ('organe', 0.4087005853652954), ('defenseur', 0.408004492521286), ('college', 0.40521830320358276)]


In [74]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('manufacturiere', 0.3887120485305786), ('rruxelles', 0.38367435336112976), ('berne', 0.37505900859832764), ('liege', 0.36691057682037354), ('thudinie', 0.3605770170688629), ('nivelles', 0.360552042722702), ('verviers', 0.3577989637851715), ('huy', 0.3493027985095978), ('xelles', 0.34713125228881836), ('dette_active', 0.34653908014297485)]


In [75]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))

[('londres', 0.5025566220283508), ('liege', 0.4976400136947632), ('gand', 0.49677982926368713), ('ostende', 0.47121167182922363), ('manchester', 0.46761396527290344), ('amsterdam', 0.4648502469062805), ('lille', 0.4556922912597656), ('hambourg', 0.436231791973114), ('marseille', 0.4356374144554138), ('verviers', 0.4276580810546875)]


In [76]:
print(model.wv.most_similar(positive=['homme', 'fidelite'], negative=['femme']))

[('obeissance', 0.5646178126335144), ('peuple_belge', 0.5225080251693726), ('<<_je_jure', 0.5205273628234863), ('serment_suivant', 0.4406374394893646), ('je_jure_fidelite', 0.42930683493614197), ('bon_sens', 0.4198843538761139), ('solennel_hommage', 0.4143155813217163), ('attachement', 0.40777382254600525), ('juste_hommage', 0.40399980545043945), ('serment_constitutionnel', 0.40388408303260803)]


In [78]:
print(model.wv.most_similar(positive=['femme', 'fidelite'], negative=['homme']))

[('<<_je_jure', 0.5334307551383972), ('obeissance', 0.5174692273139954), ('je_jure_fidelite', 0.5028979778289795), ('serment_suivant', 0.425331175327301), ('serment_constitutionnel', 0.41269707679748535), ('famille_royale', 0.3903077244758606), ('jure_fidelite', 0.38963350653648376), ('cousine', 0.3626801073551178), ('fille', 0.3621591627597809), ('peuple_belge', 0.3595965504646301)]


### Modèle 16 : Vecteur 300, fenêtre 10

In [79]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=10, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=2, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

CPU times: user 2h 5min 22s, sys: 6min 26s, total: 2h 11min 48s
Wall time: 38min 53s


In [80]:
outfile = f"../data/bulletins_16.model"
model.save(outfile)

In [81]:
model = Word2Vec.load("../data/bulletins_16.model")

In [82]:
model.wv["bruxelles"]

array([-1.0579622e+00, -5.6169719e-01,  1.1898893e+00, -3.8814965e-01,
        2.2468174e-01, -1.8987186e+00, -6.1882597e-01,  4.1828978e-01,
       -2.8047802e+00,  3.3881884e+00,  9.3040264e-01,  2.7897813e+00,
        4.0321007e-01,  1.6594656e+00,  5.3586769e-01, -1.0323501e+00,
        1.4026655e+00, -5.4061049e-01, -1.3939899e+00,  1.8595953e-01,
        3.8109545e-02,  1.4204879e-01,  1.7486145e+00, -1.0764769e+00,
       -2.5238070e+00, -9.3505830e-01,  2.0750864e+00, -1.4975587e+00,
        1.8201261e+00, -1.4629894e+00,  1.7497315e+00, -2.8160000e+00,
        7.1919709e-01,  1.1690323e+00,  2.8720286e+00, -1.9213365e+00,
       -2.9598197e-01, -3.0847213e+00,  1.3518298e+00,  1.7768536e+00,
       -1.1930108e-02,  1.6791235e+00,  8.1502736e-01, -1.2119979e+00,
       -2.0136635e+00, -2.9949892e+00, -1.5466478e+00,  2.2385702e-02,
        1.9664036e+00, -3.3665806e-02, -3.4102303e-01,  3.4911290e-01,
       -2.1772516e-01, -5.7361478e-01,  6.8714750e-01,  4.0961426e-01,
      

In [83]:
model.wv.similarity("boucher", "boulanger")

0.594906

In [84]:
model.wv.similarity("voiture", "carrosse")

0.22808711

In [85]:
model.wv.most_similar("bruxelles", topn=10)

[('nivelles', 0.5170215368270874),
 ('rruxelles', 0.5137457251548767),
 ('liege', 0.509574294090271),
 ('manufacturiere', 0.4940529763698578),
 ('tournai', 0.47900691628456116),
 ('prague', 0.47698986530303955),
 ('liruxelles', 0.47526562213897705),
 ('lille', 0.4690064787864685),
 ('xelles', 0.46365970373153687),
 ('huy', 0.4627675414085388)]

In [86]:
model.wv.most_similar("boucher", topn=10)

[('cabaretier', 0.611894428730011),
 ('charcutier', 0.6005944609642029),
 ('boulanger', 0.5949059128761292),
 ('serrurier', 0.58980792760849),
 ('marchand', 0.5820729732513428),
 ('tapissier', 0.5787808299064636),
 ('menuisier', 0.5779645442962646),
 ('tailleur', 0.576256275177002),
 ('coiffeur', 0.5692282319068909),
 ('camionneur', 0.567164957523346)]

In [87]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

[('lille', 0.4911900460720062), ('liege', 0.4882427453994751), ('londres', 0.4805184006690979), ('hambourg', 0.4721403121948242), ('huy', 0.46542081236839294), ('ostende', 0.46507900953292847), ('prague', 0.4622458219528198), ('tournai', 0.45100125670433044), ('nivelles', 0.446857213973999), ('verviers', 0.4414343535900116)]


In [88]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

[('peuple', 0.45256468653678894), ('congres_national', 0.4514980912208557), ('prince', 0.44521579146385193), ('souverain', 0.43910712003707886), ('roi_albert', 0.4365619122982025), ('magistrat', 0.4359917640686035), ('citoyen', 0.43517181277275085), ('roi_guillaume', 0.43415313959121704), ('solennel_hommage', 0.4326202869415283), ('gouvernement', 0.43144482374191284)]


In [89]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

[('conseiller', 0.43390145897865295), ('defenseur', 0.4318047761917114), ('scrupule', 0.42873626947402954), ('collegue', 0.4235033392906189), ('ministre', 0.42178773880004883), ('juriste', 0.4213380813598633), ('citoyen', 0.4180642068386078), ('geste', 0.41071468591690063), ('homme_politique', 0.4049619734287262), ('premier_ministre', 0.40486839413642883)]


In [90]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

[('nivelles', 0.39674824476242065), ('manufacturiere', 0.39540737867355347), ('rruxelles', 0.39506372809410095), ('prague', 0.3835618197917938), ('liege', 0.3790830373764038), ('termonde', 0.3711857497692108), ('tournai', 0.36896225810050964), ('geneve', 0.3603479862213135), ('berne', 0.3585074245929718), ('oflert', 0.3570314943790436)]


In [91]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))

[('londres', 0.5095242857933044), ('gand', 0.4988791048526764), ('liege', 0.49003878235816956), ('ostende', 0.48811212182044983), ('lille', 0.48261022567749023), ('hambourg', 0.45355549454689026), ('amsterdam', 0.43955713510513306), ('prague', 0.4376818537712097), ('tournai', 0.43189409375190735), ('bruges', 0.4308760166168213)]


In [92]:
print(model.wv.most_similar(positive=['homme', 'fidelite'], negative=['femme']))

[('obeissance', 0.5594314932823181), ('<<_je_jure', 0.5141829252243042), ('je_jure_fidelite', 0.4372416138648987), ('peuple_belge', 0.42120975255966187), ('sol_natal', 0.414111852645874), ('serment_suivant', 0.4088107645511627), ('solennel_hommage', 0.4080139696598053), ('citoyen', 0.4011800289154053), ('respect', 0.39841872453689575), ('attachement', 0.3957091271877289)]


In [93]:
print(model.wv.most_similar(positive=['femme', 'fidelite'], negative=['homme']))

[('obeissance', 0.49036070704460144), ('<<_je_jure', 0.4824706017971039), ('je_jure_fidelite', 0.4782809019088745), ('clause_relative', 0.43898582458496094), ('famille_royale', 0.3985356092453003), ('constance', 0.3898252844810486), ('serment_constitutionnel', 0.3866255283355713), ('tranquillite', 0.3846322000026703), ('tartine', 0.3830212354660034), ('sirene', 0.38232743740081787)]


### Modèle 17 : Vecteur 300, fenêtre 13

In [ ]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=300, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=13, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=1, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=10 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descende de gradient, aka. epochs.
)

In [ ]:
outfile = f"../data/bulletins_17.model"
model.save(outfile)

In [ ]:
model = Word2Vec.load("../data/bulletins_17.model")

In [ ]:
model.wv["bruxelles"]

In [ ]:
model.wv.similarity("boucher", "boulanger")

In [ ]:
model.wv.similarity("voiture", "carrosse")

In [ ]:
model.wv.most_similar("bruxelles", topn=10)

In [ ]:
model.wv.most_similar("boucher", topn=10)

In [ ]:
print(model.wv.most_similar(positive=['bruxelles', 'france'], negative=['belgique']))

In [ ]:
print(model.wv.most_similar(positive=['homme', 'roi'], negative=['femme']))

In [ ]:
print(model.wv.most_similar(positive=['homme', 'bourgmestre'], negative=['femme']))

In [ ]:
print(model.wv.most_similar(positive=['bourgmestre', 'bruxelles'], negative=['echevin']))

In [ ]:
print(model.wv.most_similar(positive=['bruxelles', 'paris'], negative=['belgique']))

In [ ]:
print(model.wv.most_similar(positive=['homme', 'fidelite'], negative=['femme']))

In [ ]:
print(model.wv.most_similar(positive=['femme', 'fidelite'], negative=['homme']))

In [ ]:
print(model.wv.most_similar(positive=['femme', 'homme'], negative=['fidelite']))